In [17]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
import pandas as pd

# %load_ext blackcellmagic

In [18]:
sub = pd.read_csv("sample_submission.csv")
print(sub.head())

test_data = pd.read_csv("test.csv")
test_data.head(1).values

   ImageId  Label
0        1      0
1        2      0
2        3      0
3        4      0
4        5      0


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,  10,  17,  17,  17,  17,  81, 180, 180,
         35,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0, 139, 253, 253, 253, 253, 253,
        253, 253,  48,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,  60

# tensor dataset

In [19]:
from torch.utils.data import Dataset, DataLoader
import numpy as np

class MnistDataset(Dataset):
    
    def __init__(self, x, y=None, transform=None):
        self.data = x
        self.labels = y
        self.transform = transform
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        if self.labels is not None:
            return self.transform(self.data[idx]), self.labels[idx]
        else:
            return self.transform(self.data[idx])


x = test_data.values.astype(np.float32)
x = x.reshape([-1, 28, 28, 1])

test_dataloader = DataLoader(MnistDataset(x, None, transform=transforms.Compose([transforms.ToTensor()])), batch_size=200, shuffle=False)
print(len(test_dataloader.dataset))

28000


# predict

In [20]:
class FiveLayerCNN(nn.Module):

    def __init__(self):
        super(FiveLayerCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer3 = nn.Dropout2d(0.5)
        self.layer4 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
#         self.layer5 = nn.Sequential(
#             nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=2),
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#         )
        self.fc1 = nn.Linear(576, 64)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
#         out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
#         print(out.size())
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out
        
    
m = FiveLayerCNN()
o = m(torch.rand(1, 1, 28, 28))
print(o.size())

torch.Size([1, 10])


In [21]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc1 = nn.Linear(7 * 7 * 32, 128)
        self.dropout = nn.Dropout()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

In [22]:
model = CNN().cuda()
model.load_state_dict(torch.load("mnist_cnn.pt"))
model.eval()

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1568, out_features=128, bias=True)
  (dropout): Dropout(p=0.5)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [23]:
import numpy as np
import torchvision.transforms.functional as vF
import itertools

sub = {"ImageId": [], "Label": []}

ys = []
with torch.no_grad():
    for i, data in enumerate(test_dataloader):
        data = data.cuda()
        output = model(data)
        y = output.max(1, keepdim=True)[1]  # get the index of the max log-probability
        ys.append(y.data.cpu().numpy())

merged = list(itertools.chain(*ys)) # flatten list
Label = []

for x in merged:
    Label.append(x[0])
ImageId = [t for t in range(1,len(Label)+1)]
print(len(ImageId), len(Label))
assert len(Label) == len(ImageId)
data = {
    'ImageId':ImageId,
    'Label':Label
}
df = pd.DataFrame(data=data)
df.to_csv(path_or_buf='submission.csv',index=False)

28000 28000


In [24]:
sub_df = pd.read_csv("submission.csv")
print(len(sub_df))
sub_df.head()

28000


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
